In [4]:
# Importing required packages

import warnings
warnings.filterwarnings('ignore')

from datetime import datetime
import pandas as pd
import numpy as np
import pymysql
from sqlalchemy import create_engine
import re
import time
import datetime
from datetime import datetime
import socket
import getpass

#required packages
import mysql.connector as connection
from functools import reduce
import json
import os

In [5]:
# Load credentials from the JSON file
with open('db_credentials_trg.json') as f:
    credentials = json.load(f)

# Extract the credentials
p_host = credentials['host']
p_user = credentials['user']
p_password = credentials['password']
p_database = credentials['database']

In [6]:
st = []
ed = []
du = []
date_range= []
v_host_name = []

pd.set_option('display.max_column', None)

start = datetime.now()
st.append(start)

## Final Function
def f_baseline_survey(p_start_date, p_end_date):
    
    try:
        v_visit_started_date = p_start_date
        v_visit_ended_date = p_end_date
        
        def f_connect_to_database(p_host, p_user, p_password, p_database):
            while True:
                try:
                    # Setup database connection
                    v_mydb = connection.connect(
                        host=p_host,
                        user=p_user,
                        password=p_password,
                        database=p_database,
                        use_pure=True
                    )
                    v_cursor = v_mydb.cursor()
                    # print("Connected to the database\n")
                    return v_mydb, v_cursor

                except connection.Error as err:
                    print(f"Error: {err}")
                    print("\n retry !")
                    return None, None


        # Function to close the database connection
        def f_close_database_connection(p_mydb, p_cursor):
            try:
                if p_cursor is not None:
                    p_cursor.close()
                    # print("Cursor closed.")
                if p_mydb.is_connected():
                    p_mydb.close()
                    # print("Database connection closed.\n")
            except Exception as e:
                return f"An error occurred while closing the database connection: {e}"

        while True:
            v_mydb,v_cursor = f_connect_to_database(p_host, p_user, p_password, p_database)

            v_village = input("Enter Village Id: ")
            v_state = input("Enter State Id: ")
            v_district = input("Enter District Id: ")
            v_sanch = input("Enter Sanch Id: ")
            # v_visit_speciality = input("Enter the visit_speciality value: ")

            # Update variables based on input
            s_loc_village = 'and l.location_id = ' + v_village if v_village else 'and 1=1'
            s_loc_state = 'and lst.location_id = ' + v_state if v_state else 'and 1=1'
            s_loc_district = 'and ld.location_id = ' + v_district if v_district else 'and 1=1'
            s_loc_sanch = 'and ls.location_id = ' + v_sanch if v_sanch else 'and 1=1'
            # s_visit_speciality = 'and va.value_reference = "' + v_visit_speciality + '"' if v_visit_speciality else 'and 1=1'

            if v_mydb is not None:
                #Close the database connection
                f_close_database_connection(v_mydb, v_cursor)
                break
        # Query with location filters
        v_query_concat = {'Query1':'''SELECT pi.patient_id AS Patient_id,
                    l.name as Location_village,
                    ls.name AS Location_sanch,
                    ld.name AS Location_district,
                    l.state_province as Location_state
                    FROM patient_identifier pi
                    LEFT JOIN location l ON (pi.location_id=l.location_id)
                    LEFT JOIN location ls on (ls.location_id = l.parent_location)
                    LEFT JOIN location ld on (ld.location_id = ls.parent_location )
                    LEFT JOIN location lst on (lst.location_id = ld.parent_location )
                    WHERE pi.voided=0  
                    AND pi.identifier_type = 3
                    AND pi.date_created BETWEEN %s AND %s''',
                'Query2':s_loc_village+' ',
                'Query3':s_loc_state+' ',
                'Query4':s_loc_district+' ',
                'Query5':s_loc_sanch+' ',
                'Query6':'''GROUP BY pi.patient_id, l.name, ls.name, ld.name, l.state_province;'''} 

        f_query = ''
        for query_name, query in v_query_concat.items():
            f_query = f_query + ' '+ query
        
        # print(f_query) 
                          
        # Query 1
        v_queries = {'Query1': f_query,
                     
                     'Query2':'''SELECT pat.patient_id AS Patient_id,
                              pat.date_created AS Patient_creation_date,
                              u.username AS CHW_name
                              FROM patient pat
                              JOIN users u ON (u.user_id = pat.creator)
                              WHERE pat.voided=0
                              AND pat.date_created BETWEEN %s AND %s;''',
                     
                     'Query3':'''SELECT pat.patient_id AS Patient_id,
                            MAX(pi.identifier) AS OpenMRS_id
                            FROM patient pat
                            JOIN patient_identifier pi ON (pat.patient_id = pi.patient_id)
                            LEFT JOIN location l ON (pi.location_id = l.location_id)
                            WHERE pat.voided = 0
                            AND pi.identifier_type = 3
                            AND pi.voided = 0
                            AND pat.date_created BETWEEN %s AND %s
                            GROUP BY pat.patient_id;''',
                             
                     'Query4':'''SELECT pat.patient_id AS Patient_id,
                            p.gender AS Gender,
                            p.birthdate AS Date_of_birth,
                            YEAR(CURDATE()) - YEAR(p.birthdate) AS Age
                            FROM patient pat
                            JOIN person p ON (pat.patient_id=p.person_id)
                            WHERE pat.voided=0
                            AND p.voided=0
                            AND pat.date_created BETWEEN %s AND %s;''',
                             
                     'Query5':'''SELECT pat.patient_id AS Patient_id,
                            pn.given_name AS Given_name,
                            pn.middle_name AS Middle_name,
                            pn.family_name AS Family_name
                            FROM patient pat
                            JOIN person_name pn ON (pat.patient_id=pn.person_id)
                            WHERE pat.voided=0
                            AND pn.voided=0
                            AND pn.preferred=1
                            AND pat.date_created BETWEEN %s AND %s;''',
                     
                     'Query6':'''SELECT pat.patient_id AS Patient_id,
                        	(la.value_reference) as Village_Type
                        	FROM patient pat 
                            left join patient_identifier pi on pat.patient_id = pi.patient_id
                        	LEFT JOIN location l ON (pi.location_id = l.location_id)
                        	LEFT JOIN location_attribute la ON (la.location_id = l.location_id)
                        	WHERE pat.voided = 0
                            and pi.voided = 0
                        	AND la.voided = 0
                        	AND la.attribute_type_id = 11
                        	AND pat.date_created BETWEEN %s AND %s
                        	GROUP BY pat.patient_id, la.value_reference;''',
                             
                     'Query7':'''SELECT pat.patient_id AS Patient_id,
                            pa.value,
                            pa.person_attribute_type_id
                            FROM patient pat
                            JOIN person_attribute pa ON (pat.patient_id=pa.person_id AND pa.person_attribute_type_id IN (13,15,25,26,27,55,56,8,24))
                            WHERE pat.voided=0
                            AND pa.voided=0
                            AND pat.date_created BETWEEN %s AND %s;'''}
        
        # Here append all the quries
        #v_query2 = [q1,q2,q3,q4,q5,q6]

        def f_get_date_range_from_tables(p_mydb, p_cursor):
            try:
                if p_mydb.is_connected():
                    # SQL query to fetch the minimum and maximum dates from "visit" table
                    date_query = "SELECT MIN(date_Started) AS min_date, MAX(date_Started) AS max_date FROM visit WHERE voided = 0;"

                    p_cursor.execute(date_query)
                    v_date_results = p_cursor.fetchall()

                    if v_date_results:
                        v_min_date, v_max_date = v_date_results[0]

                        return v_min_date, v_max_date

                    else:
                        print("No data available in the tables.")
                        return None, None
                else:
                    print("Database connection is not opened.")
                    return None, None
            except Exception as e:
                print(f"An error occurred: {e}")


        # Connect to the database
        v_mydb, v_cursor = f_connect_to_database(p_host, p_user, p_password, p_database)

        # Get the date range from the SQL tables
        v_min_table_date, v_max_table_date = f_get_date_range_from_tables(v_mydb, v_cursor)

        # Close the database connection
        f_close_database_connection(v_mydb, v_cursor)

        # Print the date range to inform the user
        print(f"Date range available in the SQL table: {v_min_table_date} to {v_max_table_date}\n")

        # Function to execute a query with date input parameters
        def f_execute_query_with_params(p_mydb, p_cursor, p_query, p_params):
            try:
                if p_mydb.is_connected():
                    p_cursor.execute(p_query, p_params)
                    v_results = p_cursor.fetchall()
                    v_columns = [desc[0] for desc in p_cursor.description]
                    df = pd.DataFrame(v_results, columns=v_columns)

                    return df
                else:
                    print("Database connection is not opened.")
                    return None
            except Exception as e:
                print(f"An error occurred: {e}")
                return None
            
        # Creating a function for fetching the data
        def f_fetch_data1(p_query, p_params):
            v_dataframe = []
            try:
                v_params = p_params
                
                #connect database and cursor     
                v_mydb, v_cursor = f_connect_to_database(p_host, p_user, p_password, p_database)
                    
                # Execute each query with input parameters and store the output in a dictionary
                v_output_dataframes = {}
                for query_name, query in v_queries.items():
                    # print(f"Executing {query_name}:\n{query}\nWith parameters: {v_params}\n")
                    
                    if query_name == 'Query7':
                        df = f_execute_query_with_params(v_mydb, v_cursor, query, v_params)
                        if df is not None:
                            # Pivot the data to get each attribute value into separate columns
                            pivoted_data = df.pivot(index='Patient_id', columns='person_attribute_type_id', values='value').reset_index()

                            # Rename columns for better understanding
                            pivoted_data.rename(columns={8: 'Telephone', 13: 'Education',
                                                          15: 'Occupation', 25: 'Bank_account',
                                                          26: 'Mobile_type', 27: 'Whatsapp',
                                                          55: 'HOH_relation', 56: 'Marital_status', 24: 'HH_id'}, inplace=True)

                            # Group by patient_id and aggregate values for each column
                            # The 'first' function simply selects the first value encountered within each group.
                            grouped_data = pivoted_data.groupby('Patient_id').agg({'Telephone': 'first', 'Education': 'first',
                                                                                   'Occupation': 'first', 'Bank_account': 'first',
                                                                                   'Mobile_type': 'first', 'Whatsapp': 'first',
                                                                                   'HOH_relation': 'first', 'Marital_status': 'first',
                                                                                   'HH_id': 'first'}).reset_index()
                            # Merge the grouped data with the original DataFrame on 'Patient_id'
                            merged_data = pd.merge(df['Patient_id'], grouped_data, on='Patient_id', how='inner')
                            
                            # Drop the duplicates
                            merged_data = merged_data.drop_duplicates()

                            v_dataframe.append(merged_data)
                            # print(f'{query_name} Executed successfully...\n')
                            # print(merged_data)
                        
                    else:
                        df = f_execute_query_with_params(v_mydb, v_cursor, query, v_params)
                        if df is not None:
                            v_dataframe.append(df)
                            # print(f"{query_name} executed successfully.\n")
                            # print(df)

                # Close the database connection
                f_close_database_connection(v_mydb, v_cursor)

                '''# Merge all the data frames based on 'visit_id' using 'outer' join
                merged_df = None
                for df in v_output_dataframes.values():
                    if merged_df is None:
                        merged_df = df
                    else:
                        merged_df = pd.merge(merged_df, df, on='Visit_id', how='outer')'''
                
                if len(v_dataframe)==1:
                    return v_dataframe[0]
                elif len(v_dataframe) == 2:
                    # Merge DataFrames from the list
                    v_data = pd.merge(v_dataframe[0], v_dataframe[1], on='Patient_id', how='inner')  # Merge the first two DataFrames
                    return v_data
                else:
                    v_data = pd.merge(v_dataframe[0], v_dataframe[1], on='Patient_id', how='inner')
                    # Continue merging with the rest of the DataFrames
                    for v_df in v_dataframe[2:]:
                        v_data = pd.merge(v_data, v_df, on='Patient_id', how='left')
                    return v_data
                
            except Exception as e:
                print('In this function', e)

        # Funciton for filtering the data between two days
        def f_filter_data(p_start_date, p_end_date, p_query):
            v_queries = []

            v_start_date1 = p_start_date
            v_end_date1 = p_end_date

            for i in range(len(p_query)):
                v_dummy = p_query[i].format(v_start_date1, v_end_date1)
                v_queries.append(v_dummy)

            v_data = f_fetch_data1(v_queries)
            return v_data
        
        ## SQL query to fetch the minimum and maximum dates from "visit" table
        v_min_date = v_min_table_date
        v_last_date = v_max_table_date
        v_hours = v_min_date.hour
        v_minutes = v_min_date.minute
        v_seconds = v_min_date.second

        # print(f'Date Between **Starting Date - ({v_min_date}) and **End Date - ({v_last_date}).\n\n**The format is yyy-mm-dd\n')

        try:
            while True:
                # Getting input from user
                v_strt_date_input = v_visit_started_date

                pd.to_datetime(v_strt_date_input, format='%Y-%m-%d')
                #v_start_date = v_strt_date_input+' '+str(v_hours)+':'+str(v_minutes)+':'+str(v_seconds)
                v_start_date = v_strt_date_input+' 00:00:00'
                date_format = '%Y-%m-%d %H:%M:%S'

                # Convert the string to a datetime object
                datetime_object = datetime.strptime(v_start_date, date_format)
                # print("\nDate Entered: \t",datetime_object)

                try:
                    if (datetime_object<=v_last_date): # (datetime_object>=v_min_date) and 
                        while True:
                            # Getting input from user
                            v_end_date_input = v_visit_ended_date
                            pd.to_datetime(v_end_date_input, format='%Y-%m-%d')
                            v_end_date = v_end_date_input+' 23:59:59'

                            #v_end_date = v_end_date+' 00:00:00'
                            # Convert the string to a datetime object
                            datetime_object1 = datetime.strptime(v_end_date, date_format)
                            # print("\nDate Entered: \t",datetime_object1,'\n')

                            try:
                                if (datetime_object1<v_last_date) or (datetime_object1>datetime_object):
                                    if datetime_object1 < datetime_object:
                                        print('Incorrct Date. Please Enter the Date within the range.')
                                    else:
                                        # Getting the username
                                        v_hostname = socket.gethostname()
                                        v_host_name.append(v_hostname)
                                        
                                        # Execute each query with input parameters
                                        v_params = (v_start_date, v_end_date)
                                                                                
                                        # Entered data ranges
                                        date_range.append(v_start_date+' / '+v_end_date)
                                        
                                        # Passing the startdate, end date and query in the below function to fetch the data
                                        v_sample = f_fetch_data1(v_queries, v_params)
                                        v_sample = v_sample.drop_duplicates()
                                        # print(v_sample)
                                        
                                        # Rearrange the dataframe columns as per the report file
                                        v_sample = v_sample[['Patient_id','OpenMRS_id','Location_village','Location_sanch','Location_district','Location_state','Village_Type',
                                                             'Patient_creation_date','CHW_name','Given_name','Middle_name','Family_name','Gender','Date_of_birth','Age','HH_id','Education',
                                                             'Occupation','Bank_account','Mobile_type','Whatsapp','HOH_relation','Marital_status','Telephone']].copy()
                                        
                                        # print('\nBaseline Survey dataframe created\n')
                                        
                                        # Get the current date and time
                                        file_name = datetime.now()
                                        # Format the datetime object
                                        formatted_file_name = file_name.strftime("%Y-%m-%d_%H-%M-%S")

                                        # Conver the dataframe into an excel file
                                        v_sample.to_excel(f'Baseline_survey_report_{formatted_file_name}.xlsx', index=False)
                                        
                                        print(f'\nFinal Baseline_survey_report_{formatted_file_name}.xlsx Excel Output Created...:)\n')
                                        
                                        end = datetime.now()
                                        ed.append(end)

                                        # duration = ed[-1]-st[-1]
                                        # du.append(duration)

                                        if ed[-1] is not None and st[-1] is not None:
                                            duration = ed[-1] - st[-1]
                                            du.append(duration)
                                        else:
                                            du.append(None)

                                        # Find the length of the longest list
                                        max_length = max(len(v_host_name), len(date_range), len(st), len(ed), len(du))
                                        
                                        # Extend the shorter lists with None values
                                        v_host_name.extend([None] * (max_length - len(v_host_name)))
                                        date_range.extend([None] * (max_length - len(date_range)))
                                        st.extend([None] * (max_length - len(st)))
                                        ed.extend([None] * (max_length - len(ed)))
                                        du.extend([None] * (max_length - len(du)))

                                        dur = {'Host_name':v_host_name, 'Date_Range':date_range, 'Start': st, 'End':ed, 'Duration':du}
                                        duration_df = pd.DataFrame(dur)

                                        # Convert the 'Duration' column to seconds
                                        duration_df['Duration'] = duration_df['Duration'].dt.total_seconds()
                                        # duration_df.to_excel('Duration2.xlsx', index=False)
                                        
                                        #print('Executed successfully ')
                                        
                                        # Return the file dataframe and the duration dataframe
                                        return v_sample, duration_df
                                        
                                else:
                                    print('Enter the date with range')
                            except Exception as e:
                                print('Error 2', e)
                                break
                        break
                    else:
                        print('Error3')
                    #break
                except Exception as e:
                            print(e)
        except Exception as e:
            print('Error 4', e)
    except Exception as e:
        print('Error in the main function: ', e)
        

In [7]:
v_base_survey = f_baseline_survey('2024-01-01', '2024-07-12')

Enter Village Id:  305
Enter State Id:  51
Enter District Id:  74
Enter Sanch Id:  84


Date range available in the SQL table: 2023-05-12 13:48:58 to 2024-07-12 14:55:48


Final Baseline_survey_report_2024-07-12_15-33-18.xlsx Excel Output Created...:)



In [8]:
v_base_survey[0]

,Patient_id,OpenMRS_id,Location_village,Location_sanch,Location_district,Location_state,Village_Type,Patient_creation_date,CHW_name,Given_name,Middle_name,Family_name,Gender,Date_of_birth,Age,HH_id,Education,Occupation,Bank_account,Mobile_type,Whatsapp,HOH_relation,Marital_status,Telephone
0,492736,2C62M-9,Udaypura,Dumma,Deoghar,jharkhand,Primary,2024-01-02 13:27:07,nurse1,Sudama,None,Devi,F,1964-01-02,60,d3fc0052-b661-4a8b-8b5b-5bf9ff434c0d,Cannot read or write,Household Work,Yes,Own Feature Phone,Yes (Family Member),Spouse,Married,6203927587
1,492766,2C63M-8,Udaypura,Dumma,Deoghar,jharkhand,Primary,2024-01-05 13:59:39,nurse1,Raja,None,Kumar,M,2022-01-05,2,e37be15a-809c-4b90-9f79-96a4b2bd0509,Cannot read or write,Not Applicable,Yes,Own Smartphone,Yes (Family Member),Father or Mother-In-Law,Married,6266354587
2,492767,2C63N-5,Udaypura,Dumma,Deoghar,jharkhand,Primary,2024-01-05 17:11:00,nurse1,Raja,None,Kumar,M,2019-01-05,5,ad6caab5-d48f-48f0-b16f-d1dc986a7810,Can read and write,Student,Yes,Own Feature Phone,Yes (Family Member),Father or mother,Married,9324663698
3,492768,2C63P-1,Udaypura,Dumma,Deoghar,jharkhand,Primary,2024-01-05 17:22:48,nurse1,Kashi,None,Kumar,M,2004-01-05,20,9e0fc45a-24f2-40a2-843a-1a1c56687387,Passed secondary school (Class 10th boards),Household Work,Yes,Own Smartphone,Yes (Personal),Father or mother,Unmarried,6274506687
4,492769,2C63R-7,Udaypura,Dumma,Deoghar,jharkhand,Primary,2024-01-06 15:26:55,nurse1,Aasha,None,Kumari,F,2002-05-01,22,9038acdb-9cd3-4b8c-b79e-3100cb86c98a,Middle school (6th - 8th),Student,Yes,Shared Feature Phone,Yes (Family Member),Father or mother,Unmarried,9952536993
5,492770,2C63T-2,Udaypura,Dumma,Deoghar,jharkhand,Primary,2024-01-07 17:27:37,nurse1,Renu,None,Devi,F,1995-01-07,29,3f1fb719-421d-4548-b19c-0ed270a246d9,Primary education (Till Class 5),Household Work,Yes,Shared Feature Phone,Yes (Family Member),Father or Mother-In-Law,Married,6265358087
6,492771,2C63U-0,Udaypura,Dumma,Deoghar,jharkhand,Primary,2024-01-07 17:39:19,nurse1,Rekha,None,Kumari,F,2000-01-07,24,00d69bd5-5e30-4e63-83ff-664a6855ae16,Middle school (6th - 8th),Household Work,Yes,Own Smartphone,Yes (Family Member),Father or Mother-In-Law,Married,9801158327
7,492822,2C64Y-0,Udaypura,Dumma,Deoghar,jharkhand,Primary,2024-01-20 17:53:48,nurse1,Aita,None,Kumari,F,1998-08-19,26,0a11384e-e83a-412f-963e-5c6e837465e2,Can read and write,Household Work,Yes,Own Smartphone,Yes (Personal),Father or mother,Married,6274552369
8,492823,2C650-9,Udaypura,Dumma,Deoghar,jharkhand,Primary,2024-01-20 18:01:27,nurse1,Rani,None,Devi,F,1998-01-20,26,2aa41287-15da-42c5-93e5-8b20a0027500,Can read and write,Household Work,Yes,Own Smartphone,Yes (Family Member),Father or Mother-In-Law,Married,9955253698
9,492824,2C651-7,Udaypura,Dumma,Deoghar,jharkhand,Primary,2024-01-21 17:48:55,nurse1,Gita,None,Devi,F,1996-01-21,28,84c63da5-f95c-4ce7-b32d-2a8012339c67,Cannot read or write,Household Work,Yes,Own Smartphone,Yes (Family Member),Father or Mother-In-Law,Married,6274550863


In [9]:
df1 = v_base_survey[0].copy()

In [10]:
df1.duplicated().sum()

0

In [11]:
df1['Patient_id'].duplicated().sum()

0

In [16]:
duration = v_base_survey[1]